In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [14]:
def SkipConnection(x,l):
    x1=x
    x1=layers.Conv2D(filters=l,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)
    x1=layers.Conv2D(filters=l,kernel_size=(3,3),strides=(1,1),padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x2=layers.Add()([x,x1])
    x2=layers.ReLU()(x2)

    return x2

In [13]:
inputs=Input(shape=(224,224,1))
x=layers.Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),activation='relu',padding='same')(inputs)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
x=SkipConnection(x,64)
x=SkipConnection(x,64)
x=SkipConnection(x,64)

x=layers.Conv2D(filters=128,kernel_size=(3,3),strides=(2,2),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)
x=layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)

x=SkipConnection(x,128)
x=SkipConnection(x,128)
x=SkipConnection(x,128)

x=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(2,2),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)
x=layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)

x=SkipConnection(x,256)
x=SkipConnection(x,256)
x=SkipConnection(x,256)
x=SkipConnection(x,256)
x=SkipConnection(x,256)

x=layers.Conv2D(filters=512,kernel_size=(3,3),strides=(2,2),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)
x=layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same')(x)
x=layers.BatchNormalization()(x)
x=layers.ReLU()(x)

x=SkipConnection(x,512)
x=SkipConnection(x,512)

x=layers.GlobalAveragePooling2D()(x)
output=layers.Dense(units=10,activation='softmax')(x)
   
model = tf.keras.models.Model(inputs,output)

In [ ]:
plot_model(model,to_file='ResNet.png',show_shapes=True)

In [ ]:
import visualkeras
visualkeras.layered_view(model, scale_xy=10, legend=True)

In [17]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

#One-hot encoding
y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))


x_train = np.expand_dims(x_train, axis=-1)
x_train = tf.image.resize(x_train, [224,224]) # if we want to resize 

x_test = np.expand_dims(x_test, axis=-1)
x_test = tf.image.resize(x_test, [224,224]) # if we want to resize 

In [ ]:
#callback function
lrDecay=tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,

)

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'])

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1,batch_size=8,verbose=1)

7500/7500 [==============================] - 10613s 1s/step - loss: 0.1377 - acc: 0.9585 - val_loss: 0.0646 - val_acc: 0.9807


In [26]:

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_33 (Conv2D)             (None, 112, 112, 64  3200        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_33 (BatchN  (None, 112, 112, 64  256        ['conv2d_33[0][0]']              
 ormalization)                  )                                                           